# Creating a PostgreSQL Database using Heroku

Authors: Daniel Kim & Dae Han

**To examine a real-life project application of this notebook, click here on Dae's [capstone](https://github.com/dae-han/nyc_homeless_pop_prediction/blob/master/production/1_Data_Wrangling%2BDatabase_Construction.ipynb). He built a time series model to forecast the total number of individuals sheltered by the NYC Department of Homeless Services (DHS).**

**First, go to heroku and sign up for a free account -  https://www.heroku.com/postgres**

**Click "Create new app"**

<img src="./images/01_create_app.jpeg" width="800" height = "900">

**Choose a descriptive and unique App name then click "create app"**
<img src="./images/02_app_name.jpeg" width="800" height = "900">

**Click on this [heroku add on link](https://elements.heroku.com/addons/heroku-postgresql) and click "Install Heroku Postgres"**
<img src="./images/03_install.jpeg" width="800" height = "900">

**Link the postgres add on to the app you just named.**
<img src="./images/04_postgres.jpeg" width="800" height = "900">
<img src="./images/05_add_provision.jpeg" width="800" height = "900">

**Click on "Heroku Postgres" and go to the settings and click on the database credentials.**
<img src="./images/06_creds.jpeg" width="800" height = "900">


**<font color="red">Note: Credentials are not permanent! There are other ways to obtain credentials, but this is just a simple example</font>**


Store the URI in a json file named `creds.json`.

The format in `creds.json` should be the following: ```{"uri" : "your-uri-credentials"}```. Take notice of the double quotations!

**<font color="red">DO NOT UPLOAD YOUR JSON FILE ONTO GITHUB! INCLUDE THE `creds.json` FILE IN THE GITIGNORE TEXT</font>**



In [8]:
# Run this if you do not already have this installed:
#!pip install psycopg2-binary

In [1]:
import json
import pandas as pd
import numpy as np
import sqlalchemy as db
import numpy as np
import random

random.seed(892) # to ensure everyone has the same random values

In [13]:
# Load your PostgreSQL credential json file to `creds_file`
creds_file = open('./creds.json', 'r')

# Read in the json
creds = json.loads(creds_file.read())

**Sign into your database:**

In [14]:
engine = db.create_engine(creds['uri']) # load in the uri to the engine

connection = engine.connect()
metadata = db.MetaData()

## Uploading your Pandas Dataframe to Postgres
Let's say we have a random dataframe named `df` that we wish to upload:

In [15]:
p_weight = [random.randint(0,100) for _ in range(10)]
p_height = [round(random.uniform(0,50),3) for _ in range(10)]
p_color = [random.choice(['red', 'brown', 'purple']) for _ in range(10)]
p_quality = [random.choice(['excellent', 'good', 'average', 'ew']) for _ in range(10)]
p_price = [random.randint(0,10) for _ in range(10)]

df = pd.DataFrame({'potato_weight': p_weight,  
              'potato_height': p_height,
              'potato_color': p_color,
              'potato_quality': p_quality,
              'potato_price': p_price
             })

df.head()

,potato_weight,potato_height,potato_color,potato_quality,potato_price
0,28,47.981,purple,good,2
1,96,10.823,purple,excellent,1
2,39,11.874,brown,excellent,3
3,85,8.422,purple,good,9
4,90,40.469,red,good,0


Notice how some columns are integers, objects, and floats!

In [147]:
df.dtypes

potato_weight       int64
potato_height     float64
potato_color       object
potato_quality     object
potato_price        int64
dtype: object

To create our SQL table, we will need to name the table, in this case, the table name is `sample_table`. We will also need to specify the name of the columns and their datatypes! The for-loop below grabs the columns in our `df` dataframe and their corresponding SQL datatypes (INT, TEXT, FLOAT). There are more SQL datatypes but these are the ones that correspond to our example dataframe.

In [16]:
# Grabbing column names and their datatypes:
columns_to_add = ''

for name, datatype in zip(df.columns, list(df.dtypes.values)): 
    if datatype == int:
        columns_to_add += f'{name} INT, '
    elif datatype == object:
        columns_to_add += f'{name} TEXT, '
    elif datatype == float:
        columns_to_add += f'{name} FLOAT, '

columns_to_add = "(" + columns_to_add.strip(', ') + ")"
columns_to_add

'(potato_weight INT, potato_height FLOAT, potato_color TEXT, potato_quality TEXT, potato_price INT)'

We have successfully grabbed the columns and their datatypes!

Now we will create our table by running our first SQL query!
```SQL
CREATE TABLE sample_table (potato_weight INT, potato_height INT, potato_color TEXT, potato_quality TEXT, potato_price INT)
```

In [17]:
query = f'CREATE TABLE sample_table {columns_to_add}'
query

'CREATE TABLE sample_table (potato_weight INT, potato_height FLOAT, potato_color TEXT, potato_quality TEXT, potato_price INT)'

For any SQL query we want to run, we will have to use `connection.execute()`

In [18]:
connection.execute(query) # execute sql query

Now we have a created an empty SQL table that has the same columns as our `df` dataframe.

#### **How to insert values into the table.**

I am going to create an array of tuples. Each tuple contains each row's values.

In [19]:
df.head(1)

,potato_weight,potato_height,potato_color,potato_quality,potato_price
0,28,47.981,purple,good,2


In [20]:
# example, this is the first row's values in tuple form.
tuple(df.loc[0].values)

(28, 47.981, 'purple', 'good', 2)

I am going to grab each row's information in tuples form in the for-loop below!

In [21]:
values_to_add = []

for pos in range(len(df)):
    values_to_add.append(tuple(df.loc[pos].values))
    
values_to_add

[(28, 47.981, 'purple', 'good', 2),
 (96, 10.823, 'purple', 'excellent', 1),
 (39, 11.874, 'brown', 'excellent', 3),
 (85, 8.422, 'purple', 'good', 9),
 (90, 40.469, 'red', 'good', 0),
 (31, 29.824, 'purple', 'excellent', 6),
 (57, 27.811, 'red', 'excellent', 7),
 (94, 11.601, 'purple', 'excellent', 1),
 (54, 13.946, 'red', 'ew', 2),
 (27, 40.285, 'purple', 'average', 8)]

Now we are going to insert each row from our `df` dataframe into our SQL `sample_table` table. The following SQL query format is:
```SQL
INSERT INTO sample_tables VALUES (57, 9.529, 'purple', 'good', 8)
```

In [22]:
for values in values_to_add:
    query = f'INSERT INTO sample_table VALUES {str(values)}'
    connection.execute(query)


Nice! We have successfully uploaded all of our data onto our Heroku postgreSQL server. Let's examine the data.

In [23]:
query = """
SELECT COUNT(*) 
FROM sample_table
"""

pd.read_sql_query(query, engine)

,count
0,10


There are 10 rows of data. Be warned that the free postgreSQL server's limit is 10,000 rows!
<img src="./images/07_limit.jpeg" width="800" height = "900">

In [24]:
# Grabbing all columns from sample_table
query = """
SELECT *
FROM sample_table
"""

In [25]:
# Storing queried table as a dataframe!
database_df = pd.read_sql_query(query, engine)

In [26]:
database_df

,potato_weight,potato_height,potato_color,potato_quality,potato_price
0,28,47.981,purple,good,2
1,96,10.823,purple,excellent,1
2,39,11.874,brown,excellent,3
3,85,8.422,purple,good,9
4,90,40.469,red,good,0
5,31,29.824,purple,excellent,6
6,57,27.811,red,excellent,7
7,94,11.601,purple,excellent,1
8,54,13.946,red,ew,2
9,27,40.285,purple,average,8


`df` and `database_df` are exactly the same!

In [27]:
all(database_df) == all(df)

True

In [74]:
# TO REMOVE TABLE FROM SERVER:
#connection.execute("DROP TABLE IF EXISTS sample_table")